<a href="https://colab.research.google.com/github/Tnjdh014/CodebaseRAG/blob/main/CodeBaseRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Install necessary libraries
!pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document

# Clone the repository
def clone_repository(repo_url):
    """Clones a GitHub repository to a temporary directory."""
    repo_name = repo_url.split("/")[-1]
    repo_path = f"/content/{repo_name}"
    Repo.clone_from(repo_url, str(repo_path))
    return str(repo_path)

path = clone_repository("https://github.com/CoderAgent/SecureAgent")
print(path)


GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/CoderAgent/SecureAgent /content/SecureAgent
  stderr: 'fatal: destination path '/content/SecureAgent' already exists and is not an empty directory.
'

In [7]:
# Install necessary libraries
!pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from google.colab import userdata
# Access the Pinecone API key stored as a secret
pinecone_api_key = userdata.get('PINECONE_API_KEY')

# Define supported file extensions and ignored directories
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h'}

IGNORED_DIRS = {'node_modules', 'venv', 'env', 'dist', 'build', '.git',
                '__pycache__', '.next', '.vscode', 'vendor'}

# Retrieve file content
def get_file_content(file_path, repo_path):
    """Get content of a single file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        rel_path = os.path.relpath(file_path, repo_path)
        return {"name": rel_path, "content": content}
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None

def get_main_files_content(repo_path: str):
    """Get content of supported code files from the local repository."""
    files_content = []
    try:
        for root, _, files in os.walk(repo_path):
            if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
                continue
            for file in files:
                file_path = os.path.join(root, file)
                if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                    file_content = get_file_content(file_path, repo_path)
                    if file_content:
                        files_content.append(file_content)
    except Exception as e:
        print(f"Error reading repository: {str(e)}")
    return files_content

file_content = get_main_files_content(path)

# Generate embeddings
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

text = "I am a programmer"
embeddings = get_huggingface_embeddings(text)

# Initialize Pinecone
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key
pc = Pinecone(api_key=pinecone_api_key)
pinecone_index = pc.Index("codebase-rag")

# Store documents in Pinecone
documents = []
for file in file_content:
    doc = Document(
        page_content=f"{file['name']}\n{file['content']}",
        metadata={"source": file['name']}
    )
    documents.append(doc)

vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    namespace="https://github.com/CoderAgent/SecureAgent"
)

# Initialize OpenAI client
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get("GROQ_API_KEY")
)

# Perform RAG query
def perform_rag(query):
    raw_query_embedding = get_huggingface_embeddings(query)
    top_matches = pinecone_index.query(
        vector=raw_query_embedding.tolist(),
        top_k=5,
        include_metadata=True,
        namespace="https://github.com/CoderAgent/SecureAgent"
    )
    contexts = [item['metadata']['text'] for item in top_matches['matches']]
    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[:10]) + "\n-------\n</CONTEXT>\n\nMY QUESTION:\n" + query
    system_prompt = """You are a Senior Software Engineer, specializing in TypeScript.
    Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response."""
    llm_response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )
    return llm_response.choices[0].message.content

# Example query
response = perform_rag("How is the JavaScript parser used?")
print(response)


<ipython-input-7-65936637b5c6>:85: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


The JavaScript parser (`JavascriptParser` class) is used in the `diffContextPerHunk` function to find the enclosing context of a patch for a specific file.

Here's a step-by-step breakdown of how it's used:

1. In the `diffContextPerHunk` function, the `file` parameter is passed to the `parser.findEnclosingContext` method, along with the `lineStart` and `lineEnd` values.
2. The `lineStart` and `lineEnd` values are calculated based on the new lines in the patch.
3. The `parser.findEnclosingContext` method takes the file contents, line start, and line end as input, and it returns the enclosing context of the patch.
4. The enclosing context is used to create a "scope strategy" array, which contains a map range key and a combined hunk for each scope.
5. The `buildingscopeString` function then uses the enclosing context to build the scope string for each hunk.
6. The scope strings are then added to the `contexts` array, which is used to build the final context patch.

Here's the relevant co